In [1]:
import os
import numpy as np
import torchio as tio
import nibabel as nib
from nilearn import image
import scipy.ndimage

In [2]:
# Specify input and output path
# For LINUX:
path ='ai4mi_project/data/segthor_train/train/'

# For WSL:
# path = '/home/{user}/ai4mi_project/data/segthor_train/train/'

In [3]:
def create_augmentation(folder_path, patients_tobe_augmented, new_patients_name, degree=2, aug='affine', fwhm=3, use_default_matrix=True, give_matrix=None, threshold=0, ncp=7, md=7.5):
    """
    Example of how the variables should look:
    folder_path (string): 'ai4mi_project/data/segthor_train/train/'
    patients_tobe_augmented (list of strings): ['03', '11, '27']
    new_patients_name (list of strings): ['41', '42', '43']
    degree can be (int): 2 (I think a two degree rotation is relastic, 1 would be good too but might be not enough of a difference for the model)    
    aug (string): "affine" or "gaussian" or "threshold" or "elastic"
    
    fwhm (int): 3 (Since the data is 3d, using 3 mm allows us to make a cube out of the filter, it allows the filter to take information from all directions)
    # NiLearn has its own function for applying Gaussian spatial smoothing to images as well. 
    # The only real difference from scipy.ndimage's gaussian_filter() function is that instead of 
    # specifying the smoothing kernel in standard deviations, we specify it in units of 
    # full width half-maximum (FWHM). This is the standard way that most neuroimaging analysis packages 
    # specify smoothing kernel size, so it is preferable to SciPy's approach. As the term implies, 
    # FWHM is the width of the smoothing kernel, in millimetres, at the point in the kernel where it is half 
    # of its maximum height. Thus a larger FWHM value applies more smoothing.
    
    use_default_matrix can be set to False and a different matrix can be given to parameter give_matrix
    
    threshold (int between -1000 and 31743): 0 (Everything below becomes zero) (Zero might be good because it creates the opposite of the orginal images)

    for elastic: (I have set them to defaults of the function)
    ncp (int >= 4) Number of control points to create the grid for deformation
    md (int) Maximum displacement in each direction
    """
    if use_default_matrix:
        theta = np.radians(degree)  # 10 degree rotation
        matrix = np.array([[np.cos(theta), -np.sin(theta), 0, 0],
                           [np.sin(theta), np.cos(theta),  0, 0],
                           [0,             0,              1, 0],
                           [0,             0,              0, 1]])
    else:
        matrix = give_matrix
           
    for patient, new_name in zip(patients_tobe_augmented, new_patients_name):
        
        ct = nib.load(folder_path + 'Patient_' + patient + '/Patient_' + patient + '.nii.gz')
        gt = nib.load(folder_path + 'Patient_' + patient + '/GT_fixed.nii.gz')

        # Creating directory for new patient
        os.makedirs(folder_path + 'Patient_' + new_name, exist_ok=True)
        
        if aug == 'affine':
            ct_data = ct.get_fdata()
            gt_data = gt.get_fdata()
            
            ct_new_img = scipy.ndimage.affine_transform(ct_data, matrix, order=1)
            gt_new_img = scipy.ndimage.affine_transform(gt_data, matrix, order=0)
            
            # Added to pass sanity check when slicing
            gt_new_img = np.rint(gt_new_img).astype(np.int16)
        
            ct_new_img = nib.Nifti1Image(ct_new_img, ct.affine, ct.header)
            gt_new_img = nib.Nifti1Image(gt_new_img, gt.affine, gt.header)

        elif aug == 'gaussian':
            ct_new_img = image.smooth_img(ct, fwhm)
            gt_new_img = gt

        elif aug == 'threshold':
            data = ct.get_fdata() 
            data[data < threshold] = 0
            ct_new_img = nib.Nifti1Image(data, ct.affine)
            gt_new_img = gt

        elif aug == 'elastic':
            ct_path = os.path.join(folder_path + 'Patient_' + patient + '/Patient_' + patient + '.nii.gz')
            gt_path = os.path.join(folder_path + 'Patient_' + patient + '/GT_fixed.nii.gz')
            
            ct_el = tio.Subject(image=tio.ScalarImage(ct_path))
            gt_el = tio.Subject(image=tio.LabelMap(gt_path))  # Using LabelMap for segmentation labels

            # Define the elastic transformation
            # https://torchio.readthedocs.io/_modules/torchio/transforms/augmentation/spatial/random_elastic_deformation.html#furo-main-content
            elastic = tio.RandomElasticDeformation(
                num_control_points=(ncp, ncp, ncp),  # Number of control points to create the grid for deformation
                max_displacement=(md, md, md)  # Maximum displacement in each direction
            )

            # Create a subject that contains both CT and GT to ensure consistent transformation
            subject = tio.Subject(
                ct_image=ct_el['image'],  # Scalar image for the CT
                gt_image=gt_el['image']  # Label map for the GT
            )

            transformed_subject = elastic(subject)

            ct_new_img = transformed_subject['ct_image']
            gt_new_img = transformed_subject['gt_image']
            
            ct_new_img = ct_new_img.data.numpy().squeeze().astype(np.float64)
            gt_new_img = gt_new_img.data.numpy().squeeze().astype(np.uint8)
            
            ct_new_img = nib.Nifti1Image(ct_new_img, ct.affine, ct.header)
            gt_new_img = nib.Nifti1Image(gt_new_img, gt.affine, gt.header)
            
        # Save the transformed image
        nib.save(ct_new_img, folder_path + 'Patient_' + new_name + '/Patient_' + new_name + '.nii.gz')
        nib.save(gt_new_img, folder_path + 'Patient_' + new_name + '/GT_fixed.nii.gz')

# affine:
patients_tobe = ['01', '12', '24', '35', '40']
new_patients = ['41', '42', '43', '44', '45']
create_augmentation(path, patients_tobe, new_patients, aug='affine')
# gaussian:
patients_tobe = ['03', '15', '23', '36', '13']
new_patients = ['46', '47', '48', '49', '50']
create_augmentation(path, patients_tobe, new_patients, aug='gaussian')
# threshold:
patients_tobe = ['06', '11', '20', '38', '02']
new_patients = ['51', '52', '53', '54', '55']
create_augmentation(path, patients_tobe, new_patients, aug='threshold')
# elastic:
patients_tobe = ['09', '19', '25', '33', '27']
new_patients = ['56', '57', '58', '59', '60']
create_augmentation(path, patients_tobe, new_patients, aug='elastic')